In [1]:
from platypus import NSGAII, Problem, Real
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.colors as mcolors
import pandas as pd

In [2]:
# Load the autoreload extension
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

In [3]:
from Mult_Obj_Helathy import Main_Model as Mdl_H 
from Mult_Obj_Tumor import Main_Model as Mdl_T

# Mixture of CAR NK and WT
(1) check for 50 and 50 

In [4]:
Shell_H = Mdl_H()
Shell_T = Mdl_T()

In [5]:
Tumor_No = 10000
for i in [10.0,5.0,2.5,1.25,0.625,0.3125]:
    x0 = i
    NK_No = x0*Tumor_No
    y0 = 1.0
    x = [0.9791789663709559,0.8936323663149114]
    frac = 0.8
    Shell_T.Tumor(x[0], frac, x[1], NK_No=NK_No, Tc_No = Tumor_No)
    Shell_H.Healthy(x[0], frac, x[1], NK_No=NK_No, Tc_No = Tumor_No)
    print('done')

T 53.392371004926865
H 75.60295376823717
done
T 41.94746013722439
H 62.40345581382408
done
T 31.099191650434076
H 44.96771135546004
done
T 20.869497872618325
H 28.209560514716458
done
T 12.508050858428177
H 16.00039385689085
done
T 6.91774723086882
H 8.551013356434856
done


# Tumor Cell
### (i) CAR_killing
    1/78.94391239956171 == 0.012667221190389747
### (ii) WT killing
    1/40.27411983691734 ==0.02482984119949279

# Healthy Cell
### (i) CAR_killing
    33.209108322965996
### (ii) WT killing
    7.810341768315665

In [7]:
# Define your multi-objective problem here
class MyProblem(Problem):
    def __init__(self,ET_ratio):
        super(MyProblem, self).__init__(1, 2) # #variables, #objectives)
        self.types[:] = [Real(0.001, 1.0)]
        self.Shell_H = Mdl_H()
        self.Shell_T = Mdl_T()
        self.ET_ratio = ET_ratio
        self.directions[:] = [Problem.MINIMIZE, Problem.MAXIMIZE]

    def evaluate(self, solution):
        x = np.array(solution.variables)
        print('x0',x)
        Tumor_No = 10000
        NK_No = self.ET_ratio*Tumor_No
        f1 = self.Shell_H.Healthy(0.9791789663709559, x[0], 0.8936323663149114, NK_No=NK_No, Tc_No = Tumor_No) # Objective 1
        f2 = self.Shell_T.Tumor(0.9791789663709559, x[0], 0.8936323663149114, NK_No=NK_No, Tc_No = Tumor_No) # Objective 2
        solution.objectives[:] = [f1, f2]

In [8]:
for ET_ratio in [1,5,10]:
        problem = MyProblem(ET_ratio)
        algorithm = NSGAII(problem)
        algorithm.run(2)
        Healthy_Lysis = [s.objectives[0] for s in algorithm.result]
        Tumor_Lysis = [s.objectives[1] for s in algorithm.result]
        w =  [s.variables[0] for s in algorithm.result]
        filename = f'a4_bw_p001_1_Lysis_Kasumi_Healthy_CAR_NK_a4_ET_{ET_ratio}.xlsx'
        df = pd.DataFrame(columns=['w','Tumor_lysis', 'Healthy_Lysis'])
        df['w'] = w
        df['Tumor_lysis'] = Tumor_Lysis
        df['Healthy_Lysis'] = Healthy_Lysis
        df.to_excel(filename)
        print('Done for', ET_ratio)

x0 [0.65578036]
H 20.63241946218931
T 16.694519992055223
x0 [0.16176749]
H 8.553267229643291
T 12.418585331109345
x0 [0.36190932]
H 13.827605360052376
T 14.314884678516915
x0 [0.936867]
H 26.379634756354065
T 18.853687173584355
x0 [0.09411771]
H 6.659395421324588
T 11.756687588161508
x0 [0.61555376]
H 19.80261775169858
T 16.47231378090732
x0 [0.79536689]
H 23.581749184672596
T 17.79703760870637
x0 [0.2868355]
H 11.958663233576416
T 13.711016764227068
x0 [0.24456591]
H 10.837188621607808
T 13.29238301159965
x0 [0.84483228]
H 24.5746533051897
T 18.16072465401065
x0 [0.32882217]
H 12.939696528172306
T 13.919609356311247
x0 [0.27601274]
H 11.684890857941443
T 13.624173174916343
x0 [0.33397216]
H 13.046472190828162
T 13.92323547998523
x0 [0.5188615]
H 17.6146173758988
T 15.684388717556008
x0 [0.5767878]
H 18.896492270058452
T 16.083979188857434
x0 [0.36687348]
H 13.91829030082523
T 14.2987895891031
x0 [0.86253197]
H 24.928815082853962
T 18.297222335581196
x0 [0.32110711]
H 12.82986581176903